In [9]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_Weights
from dataset_loader.dataset import DatasetConfig, ObjectDetectionDataset  # Asegúrate de que estén en el mismo directorio o en el PATH
from torchvision.ops import box_iou


In [10]:
# Cargar modelo Mask R-CNN solo para detección de objetos
weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
model = maskrcnn_resnet50_fpn(weights=weights,)
model.roi_heads.mask_predictor = None  # Remover la cabeza de predicción de máscaras

# Número de clases (incluye la clase fondo)
num_classes = 3  # 3 clases más la clase fondo
in_features = model.roi_heads.box_predictor.cls_score.in_features
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Cargar pesos personalizados si corresponde
try:
    model.load_state_dict(torch.load("custom_maskrcnn.pt"))
    print("Pesos personalizados cargados con éxito.")
except FileNotFoundError:
    print("No se encontró el archivo de pesos personalizados. Usando pesos preentrenados.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Pesos personalizados cargados con éxito.


/var/folders/9s/6cdycfcs5tb2fjsp6gqq4mgm0000gn/T/ipykernel_15726/3269224183.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("custom_mas

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [19]:
#quiero ver cuales son los parametros que no estan fijos de todo el modelo
params = [p for p in model.parameters() if p.requires_grad]
print(len(params))


80


In [12]:
#cuantos parametros hay en total
total_params = sum(p.numel() for p in model.parameters())
print(total_params)


43664606


In [13]:
#quiero liberar los parametros de la capa de deteccion de objetos
for p in model.roi_heads.box_predictor.parameters():
    p.requires_grad = True

#verificar que se hayan fijado

In [21]:
#quiero ver cuales son los parametros que no estan fijos
params = [p for p in model.parameters() if p.requires_grad]
#ver la cantidad
print(params[0].shape)

torch.Size([128, 256, 1, 1])


In [15]:
#quiero liberar mas capas

for name, params in model.backbone.named_parameters():
    if 'layer3' in name:
        print(name)
        #hacer print si esta fijo o no
        print(params.requires_grad)

        params.requires_grad = True
        print(params.requires_grad)



for p in model.roi_heads.box_predictor.parameters():
    p.requires_grad = True

#verificar que se hayan fijado



body.layer3.0.conv1.weight
True
True
body.layer3.0.conv2.weight
True
True
body.layer3.0.conv3.weight
True
True
body.layer3.0.downsample.0.weight
True
True
body.layer3.1.conv1.weight
True
True
body.layer3.1.conv2.weight
True
True
body.layer3.1.conv3.weight
True
True
body.layer3.2.conv1.weight
True
True
body.layer3.2.conv2.weight
True
True
body.layer3.2.conv3.weight
True
True
body.layer3.3.conv1.weight
True
True
body.layer3.3.conv2.weight
True
True
body.layer3.3.conv3.weight
True
True
body.layer3.4.conv1.weight
True
True
body.layer3.4.conv2.weight
True
True
body.layer3.4.conv3.weight
True
True
body.layer3.5.conv1.weight
True
True
body.layer3.5.conv2.weight
True
True
body.layer3.5.conv3.weight
True
True


In [16]:
#quiero ver cuales son los parametros que no estan fijos
params = [p for p in model.parameters() if p.requires_grad]
#ver la cantidad
print(len(params))

80
